In [ ]:
import pyodbc

class RestaurantSystem:
    def __init__(self):
        self.conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LAPTOP-UIRB00PE\SQLEXPRESS;"
            "DATABASE=proj;"
            "Trusted_Connection=yes;"
        )
        self.cursor = self.conn.cursor()


    def show_menu(self):
        self.cursor.execute("SELECT id, name, price FROM menu")
        items = self.cursor.fetchall()

        print("\n------ MENU ------")
        for item in items:
            print(f"{item.id}. {item.name} - ${item.price}")

    def generate_order_id(self):
        self.cursor.execute("SELECT ISNULL(MAX(order_id),0) + 1 FROM orders")
        return self.cursor.fetchone()[0]


    def take_order(self, table_number):
        order_id = self.generate_order_id()
        print(f"\nNew Order ID: {order_id}")

        while True:
            self.show_menu()
            choice = input("Enter menu ID (or 'done'): ")

            if choice.lower() == "done":
                break

            quantity = int(input("Enter quantity: "))

            self.cursor.execute(
                "SELECT id, name, price FROM menu WHERE id = ?",
                (choice,)
            )
            item = self.cursor.fetchone()

            if item:
                item_id, name, price = item
                total_price = float(price) * quantity

                self.cursor.execute("""
                    INSERT INTO orders 
                    (order_id, item_id, item_name, quantity, rate, total_price)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (order_id, item_id, name, quantity, price, total_price))

                self.conn.commit()
                print(f"{name} added to order.")
            else:
                print("Invalid item ID.")

        print("Order saved successfully.")


    def print_bill(self, order_id):
        self.cursor.execute("""
            SELECT item_name, quantity, rate, total_price
            FROM orders
            WHERE order_id = ?
        """, (order_id,))

        items = self.cursor.fetchall()

        if not items:
            print("Order not found.")
            return

        print(f"\n------ BILL (Order {order_id}) ------")
        grand_total = 0

        for item in items:
            print(f"{item.item_name} x{item.quantity} @ ${item.rate} = ${item.total_price}")
            grand_total += float(item.total_price)

        print("----------------------------")
        print(f"Total Amount: ${grand_total}")

In [ ]:
if __name__ == "__main__":
    system = RestaurantSystem()

    while True:
        print("\n1. Show Menu")
        print("2. Take Order")
        print("3. Print Bill")
        print("4. Exit")

        choice = input("Enter choice: ")

        if choice == "1":
            system.show_menu()

        elif choice == "2":
            table = int(input("Enter table number: "))
            system.take_order(table)

        elif choice == "3":
            order_id = int(input("Enter Order ID: "))
            system.print_bill(order_id)

        elif choice == "4":
            print("System closed. SQL Server still respects you.")
            break

        else:
            print("Invalid choice.")
